In [43]:
import pandas as pd
import numpy as np

In [44]:
file = open('../input/Thus Spoke Zarathustra.txt')

In [45]:
text = file.read()

In [46]:
text = text.replace('\r\n\r\n', '\r\n')

In [47]:
text = text.replace('\r\n', ' ')

In [48]:
text = text.replace('\'', '')

In [49]:
import re

In [50]:
text = re.sub(' +',' ',text)

In [51]:
text = text.split()

In [52]:
words = list(set(text))

In [53]:
vocab_size = len(words)

In [54]:
from scipy.sparse import csr_matrix
from scipy.sparse import vstack

In [55]:
strdict = {}
revdict = {}
i = 0
for word in words:
    row = [0]
    col = [i]
    data = [1]
    strdict[word] =csr_matrix((data, (row, col)), shape=(1, vocab_size))
    revdict[i] = word
    i += 1

In [93]:
def convert_to_df(start, size, seq, vsize, batch_size):
    word_count = len(seq)
    if batch_size == 1:
        ones = seq[start:start + size]
        inp_vector = vstack([strdict[x] for x in ones])
        out_vec = strdict[seq[start + size]]
        inp = inp_vector.toarray()
        out = out_vec.toarray()
    else:
        inp = np.array([])
        out = np.array([])
        for i in range(batch_size):
            if start >= word_count - size:
                break
            ones = seq[start:start + size]
            inp_vector = vstack([strdict[x] for x in ones])
            out_vec = strdict[seq[start + size]]
            if i == 0:
                inp = inp_vector.toarray()
                out = out_vec.toarray()
            else:
                inp = np.dstack((inp, inp_vector.toarray()))
                out = np.vstack((out, out_vec.toarray()))
            start += 1
        inp = np.swapaxes(inp, 2, 0)
        inp = np.swapaxes(inp, 1, 2)
    return inp, out

In [94]:
convert_to_df(0, 32, text, vocab_size, 1)[0].shape

(32, 16665)

In [95]:
import tensorflow as tf

In [111]:
def rnn_cell(W,Wo, U, b, prev_cell, curr):
    h = tf.tanh(tf.add(tf.matmul(U, curr), tf.matmul(W, prev_cell)) + b)
    out = tf.matmul(Wo, h)
    return h, out

In [130]:
def rnn_layer(xin, We, Wo, Ue, be, num_inp):
    layer = tf.zeros((1, vocab_size))
    next_inp = []
    for i in range(num_inp):
        curr = xin[i]
        curr = tf.cast(curr, tf.float32)
        curr = tf.reshape(curr, [1, vocab_size])
        layer = tf.cast(layer, tf.float32)
        layer, out = rnn_cell(We, Wo, Ue, be, layer, curr)
        next_inp.append(out)
    next_inp = tf.stack(next_inp)
    next_inp = tf.squeeze(next_inp)
    return next_inp

In [125]:
learning_rate = 0.0001
training_iters = 10
display_step = 1
num_inp = 64
n_hidden = 7
m = len(text)
batch_size = 1

In [126]:
x = tf.placeholder(tf.float64, [num_inp, vocab_size])
y = tf.placeholder(tf.float64, [None, vocab_size])

In [127]:
W = {}
U = {}
WO = {}
b = {}
for i in range(1, n_hidden + 1):
    W[i] = tf.Variable(tf.truncated_normal([1, 1], stddev=0.03))
    WO[i] = tf.Variable(tf.truncated_normal([1, 1],  stddev=0.03))
    U[i] = tf.Variable(tf.truncated_normal([1, 1],  stddev=0.03))
    b[i] = tf.Variable(tf.zeros([1, vocab_size]))

In [128]:
tf.device("/device:GPU:0")

In [139]:
prev_inp = x
for i in range(1, n_hidden + 1):
    prev_inp = rnn_layer(prev_inp, W[i],WO[i], U[i], b[i], num_inp)
dense_layer1 = prev_inp[-1:]

In [140]:
pred = tf.nn.softmax(dense_layer1)

In [141]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=dense_layer1, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [142]:
# Model evaluation
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [143]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    window_size = m - num_inp
    for epoch in range(training_iters):
        cst_total = 0
        acc_total = 0
        total_batches = int(np.ceil(window_size / batch_size))
        for i in range(total_batches):
            df_x, df_y = convert_to_df(i, num_inp, text, vocab_size, batch_size)
            _, cst, acc = sess.run([optimizer, cost, accuracy], feed_dict = {x : df_x, y : df_y})
            cst_total += cst
            acc_total += acc
        if (epoch + 1) % display_step == 0:
            print('After ', (epoch + 1), 'iterations: Cost = ', cst_total / total_batches, 'and Accuracy: ', acc_total * 100 / total_batches, '%' )
    print('Optimiation finished!!!')
    save_path = saver.save(sess, "../working/model.ckpt")
    print("Model saved in path: %s" % save_path)
    print("Lets test")
    sentence = '''And many a one who cannot see men s loftiness, calleth it 
virtue to see their baseness far too well: thus calleth he his 
evil eye virtue. And some want to be edified and raised up, and call it 
virtue: and others want to be cast down, and likewise call 
it virtue. And thus do almost all think that they participate in virtue; 
and'''
    sent = sentence.split()
    sent = vstack([strdict[s] for s in sent])
    sent  = sent.toarray()
    for q in range(64):
        one_hot = sess.run(pred, feed_dict={x : sent.reshape((1, num_inp, vocab_size))})
        index = int(tf.argmax(one_hot, 1).eval())
        sentence = "%s %s" % (sentence,revdict[index])
        sent = sent[1:]
        sent = np.vstack((sent, one_hot))
    print(sentence)
    